# Assessment 1: BugFinder
### Intro
This notebook is to be the record of completion for Assessment 1: Machine Learning.
### Objective
Automate identification of insects in real-time using a pre-trained image classification model and camera robot.